# Разложение Шура

Любую квадратную матрицу $A$ можно преобразованием $U$ подобия привести к верхнетреугольной матрице $R$,
причем преобразование $U$ можно выбрать унитарным (ортогональным для вещественных матриц):
$$A=URU^*,\quad U^*AU=R,\quad UU^*=U^*U=1,$$
такое представление матрицы называется [разложением Шура](https://en.wikipedia.org/wiki/Schur_decomposition).
Здесь и далее мы будем обозначать единичную матрицу подходящего размера через $1$,
а сопряженную матрицу к $A$ через $A^*$.
Элементы сопряженной матрицы получаются транспонирование матрицы с последующим комплексным сопряжением:
$$
A^*=\bar A^T,\quad (A^*)_{nk}=\overline{A_{kn}}. 
$$ 
Столбцы матрицы преобразования $U$ называются векторами Шура.

### Задания.

1. Какой смысл имеют диагональные элементы матрицы $R$?

2. В каком случае вектора Шура оказываются собственными векторами?

3. Покажите, что сумма квадратов абсолютных значений всех недиагональных элементов матрицы $R$ не зависит от выбора $U$ и определяется только матрицей $A$:
$$ N = \sum_{n}\sum_{k>n} |R_{nk}|^2.$$
Какой смысл вы можете придать этой сумме?

Первый вектор Шура всегда является собственным, поэтому мы можем построить наивную процедуру
вычисления разложения Шура, имея способ вычисления собственных чисел и векторов.
Проще всего оказывается вычислить самое большое по модулю собственное значение матрицы (спектральный радиус),
для этого можно воспользоваться [методом степеней](https://en.wikipedia.org/wiki/Power_iteration).
Суть метода заключается в вычислении последовательности:
$$e_{n+1}=\frac{Ae_n}{\|Ae_n\|},$$
которая при определенных условиях сходится к собственному вектору, отвечающему максимальному по модулю собственому значению $A$. 

## Задания.

4. Предложите достаточные условия сходимости степенного метода.

5. Реализуйте степенной метод. Для проверки результата воспользуйтесь функцией 
[scipy.linalg.norm(A, ord=2)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.norm.html),
которая на квадратной матрице $A$ возвращает ее спектральных радиус.

6. Первый вектор Шура находится степенным методом, как можно найти второй вектор Шура? 
__Указание:__ Рассмотрите подматрицу $A_{2\colon,2\colon}$.

7. Реализуйте функцию для построения разложения Шура с помощью степенного метода. В каких случаях алгоритм сойдется? В каких случаях сойдется к разложению Шура? С какой скоростью итерации сходятся в случае сходимости?

8. Обобщите степенной метод так, чтобы одновременно вычислялось несколько собственных векторов. 
Реализуйте эту модификацию. Какие условия являются достаточными для сходимости вашего метода.

Задача вычисления собственных чисел может быть плохо обусловлена для матрицы общего вида. 
Наибольшую трудность представляют близкие собственные значения и вырожденные собственные значения. 
Проведите несколько экспериментов, используя библиотечную функцию для вычисления разложения Шура 
[scipy.linalg.schur](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.schur.html).


## Задания.

9. Предложите матрицу, у которой левый и правый собственные вектра для одного собственного значения почти ортогональны. Добавляя малое возмущение к матрице (можно воспользоваться [numpy.random.randn](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)) и находя собственные значения для возмущенной матрицы через `scipy.linalg.schur` оцените число обусловленности для вычисления собственного числа. Сравните с теорией.

10. Рассмотрите малое возмущение $\epsilon$ для матрицы 
$$A=\begin{pmatrix}1&a\\\epsilon&1\end{pmatrix},$$
где $a$ - параметр. Насколько сильно возущение изменяет собственные значения?
Собственные вектора (`scipy.linalg.eig`)?

На практике для вычисления разложения Шура как правило используется [QR алгоритм](https://en.wikipedia.org/wiki/QR_algorithm) и его варианты. 
Мы ограничимся изучением этого метода только для симметричных матриц.

Пусть матрица $A$ имеет только вещественные коэффициенты и симметрична, т.е. $A^T=A$.
В этом случае матрица $R$ в разложение Шура для матрица $A$ оказывается диагональным, т.е. выполняется спектральное разложение.

Перед выполнение QR алгоритма матрица $A$ приводится преобразованием подобия к более простому виду $A_0=VAV^T$,
как правило к виду [матрицы Хессенберга](https://en.wikipedia.org/wiki/Hessenberg_matrix).
Преобразование $V$ можно представить, например, в виде цепочки [вращений Гивенса](https://en.wikipedia.org/wiki/Givens_rotation).
На одном шаге QR алгоритма строится QR разложение матрицы $A_n=Q_nR_n$,
затем матрицы из разложения перемножаются в обратном порядке, формируя новый член последовательности:
$A_{n+1}=R_nQ_n$.
Все матрицы в последовательности подобны: $A_{n+1}=Q_n^TA_nQ_n$.
Итерации повторяются до тех пор, пока матрица $A_n$ не станет достаточно треугольной.

В наивном варианте QR алгоритм не всегда сходится, однако ситуацию можно исправить, добавив в сдвиги.
На каждом шаге алгоритма будем строить QR разложение для $A_n-\zeta_n=Q_nR_n$ с подходящим $\zeta_n$.
Следующий член последовательности определим так $A_{n+1}=R_nQ_n+\zeta_n$.
Последовательность $\zeta_n$ выбирается так, чтобы $\zeta_n$ сходилось к минимальному собственному числу,
например, полагая $\zeta_n$ равным элементу $R$ из последнего столбца и строки.
В этому случае итерации почти всегда сходятся и дают кубическую скорость сходимости.

## Задания.

11. Реализуйте QR алгоритм со сдвигами для симметричной матрицы $A$. 
Экспериментально проверьте скорость сходимости. 
Сравните со скорость сходимости степенного метода.

12. **(повышенная сложность)** Реализуйте неявный QR алгоритм. Сравните его работу с работой явного метода.

13. Предложите и реализуйте метод вычисления [сингулярного (SVD) разложения](https://en.wikipedia.org/wiki/Singular_value_decomposition), используя разложение Шура. Постарайтесь избежать вычисления матриц $AA^T$ и $A^TA$.